<a href="https://colab.research.google.com/github/MRM07/EverythingDataScience/blob/master/FeatureTools_for_Relational_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import featuretools as ft

In [0]:
clients= pd.read_csv("clients.csv")
loans=   pd.read_csv("loans.csv")
payments= pd.read_csv("payments.csv")

In [82]:
clients.head()

,client_id,joined,income,credit_score
0,46109,2002-04-16,172677,527
1,49545,2007-11-14,104564,770
2,41480,2013-03-11,122607,585
3,46180,2001-11-06,43851,562
4,25707,2006-10-06,211422,621


In [84]:
loans.head()

,client_id,loan_type,loan_amount,repaid,loan_id,loan_start,loan_end,rate
0,46109,home,13672,0,10243,2002-04-16,2003-12-20,2.15
1,46109,credit,9794,0,10984,2003-10-21,2005-07-17,1.25
2,46109,home,12734,1,10990,2006-02-01,2007-07-05,0.68
3,46109,cash,12518,1,10596,2010-12-08,2013-05-05,1.24
4,46109,credit,14049,1,11415,2010-07-07,2012-05-21,3.13


In [85]:
payments.head()

,loan_id,payment_amount,payment_date,missed
0,10243,2369,2002-05-31,1
1,10243,2439,2002-06-18,1
2,10243,2662,2002-06-29,0
3,10243,2268,2002-07-20,0
4,10243,2027,2002-07-31,1


# Entity


An entity is simply a table, which is
represented in Pandas as a dataframe. 
Each entity must have a uniquely identifying column, known as an index. 


For the clients dataframe, this is the client_id because each id only appears once in the clients data. 

In the loans dataframe, client_id is not an index because each id might appear more than once. The index for this dataframe is instead loan_id.


Feature tools will automatically infer the variable types (numeric, categorical, datetime) of the columns in our data, but we can also pass in specific datatypes to override this behavior. As an example, even though the repaid column in the loans dataframe is represented as an integer, we can tell feature tools that this is a categorical feature since it can only take on two discrete values. This is done using an integer with the variables as keys and the feature types as values.

In [0]:
es = ft.EntitySet(id = 'clients')

Specify the categorical/numerical/date_time columns through a list, which we will add as a feature in our UI, where 
the user shall have an option to mark the category of the features like, date_time, numeric or categorical. And based on that,
our feature tools will perform aggregations and standardization.




In [0]:
#our categorical list here is small but let's add our data to it

categorical_list=['repaid', 'missed']

In [0]:
"""
Let's create a dictionary to define the categorical columns, because our 
categorical columns are in the form of numeric values.
This dictionary is fed in the function in the next cell tab in variable_types parameter.

"""
categorical_dict={}
for col in categorical_list:
  categorical_dict[col]= ft.variable_types.Categorical
  

In [0]:
# Create an entity from the loans dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'loans', dataframe = loans, 
                              variable_types = categorical_dict,
                              index= "loan_id"
                              )

In [0]:
# Create an entity from the client dataframe
# This dataframe already has an index and a time index
es = es.entity_from_dataframe(entity_id = 'clients', dataframe = clients, 
                              index = 'client_id', time_index = 'joined')

In [0]:
# Create an entity from the payments dataframe
# This does not yet have a unique index
es = es.entity_from_dataframe(entity_id = 'payments', 
                              dataframe = payments,
                              variable_types = {'missed': ft.variable_types.Categorical},
                              make_index = True,
                              index = 'payment_id',
                              time_index = 'payment_date')

All three entities have been successfully added to the EntitySet. We can access any of the entities using Python dictionary syntax.



Feature tools correctly inferred each of the datatypes when we made this entity. We can also see that we overrode the type for the repaid feature, changing if from numeric to categorical.

In [107]:
es['clients']

Entity: clients
  Variables:
    client_id (dtype: index)
    joined (dtype: datetime_time_index)
    income (dtype: numeric)
    credit_score (dtype: numeric)
  Shape:
    (Rows: 25, Columns: 4)

In [97]:
es['loans']

Entity: loans
  Variables:
    loan_id (dtype: index)
    client_id (dtype: numeric)
    loan_type (dtype: categorical)
    loan_amount (dtype: numeric)
    loan_start (dtype: datetime)
    loan_end (dtype: datetime)
    rate (dtype: numeric)
    repaid (dtype: categorical)
  Shape:
    (Rows: 443, Columns: 8)

In [98]:
es['payments']

Entity: payments
  Variables:
    payment_id (dtype: index)
    loan_id (dtype: numeric)
    payment_amount (dtype: numeric)
    payment_date (dtype: datetime_time_index)
    missed (dtype: categorical)
  Shape:
    (Rows: 3456, Columns: 5)

# Relationships


After defining the **entities** (tables) in an EntitySet, we now need to tell feature tools how they are related with a **relationship**. 

The most intuitive way to think of relationships is with the parent to child analogy: a parent-to-child relationship is one-to-many because for each parent, there can be multiple children.

The client dataframe is  parent of the loans dataframe because while there is only one row for each client in the client dataframe, each client may have several previous loans covering multiple rows in the loans dataframe. 


Likewise, the loans dataframe is the parent of the payments dataframe because each loan will have multiple payments.


**We codify relationships in the language of feature tools by specifying the parent variable and then the child variable. After creating a relationship, we add it to the EntitySet.**

In [0]:
# Relationship between clients and previous loans
r_client_previous = ft.Relationship(es['clients']['client_id'],
                                    es['loans']['client_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_client_previous)

The relationship has now been stored in the entity set. The second relationship is between the **loans and payments**. These two entities are related by the **loan_id** variable.

In [109]:
# Relationship between previous loans and previous payments
r_payments = ft.Relationship(es['loans']['loan_id'],
                                      es['payments']['loan_id'])

# Add the relationship to the entity set
es = es.add_relationship(r_payments)

es

Entityset: clients
  Entities:
    loans [Rows: 443, Columns: 8]
    clients [Rows: 25, Columns: 4]
    payments [Rows: 3456, Columns: 5]
  Relationships:
    loans.client_id -> clients.client_id
    payments.loan_id -> loans.loan_id

# Feature Primitives


A feature primitive a at a very high-level is an operation applied to data to create a feature. These represent very simple calculations that can be stacked on top of each other to create complex features. Feature primitives fall into two categories:

**Aggregation**: function that groups together child datapoints for each parent and then calculates a statistic such as mean, min, max, or standard deviation. An example is calculating the maximum loan amount for each client. An aggregation works across multiple tables using relationships between tables.


**Transformation**: an operation applied to one or more columns in a single table. An example would be extracting the day from dates, or finding the difference between two columns in one table.
Let's take a look at feature primitives in feature tools. We can view the list of primitives:

In [110]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)

,name,type,description
0,max,aggregation,Finds the maximum non-null value of a numeric feature.
1,min,aggregation,Finds the minimum non-null value of a numeric feature.
2,num_unique,aggregation,Returns the number of unique categorical variables.
3,std,aggregation,Finds the standard deviation of a numeric feature ignoring null values.
4,mode,aggregation,Finds the most common element in a categorical feature.
5,skew,aggregation,Computes the skewness of a data set.
6,mean,aggregation,Computes the average value of a numeric feature.
7,median,aggregation,Finds the median value of any feature with well-ordered values.
8,avg_time_between,aggregation,Computes the average time between consecutive events.
9,sum,aggregation,Sums elements of a numeric or boolean feature.


# **Deep Feature Synthesis**


Feature primitives are useful by themselves, the main benefit of using feature tools arises when we stack primitives to get deep features. The depth of a feature is simply the number of primitives required to make a feature. So, a feature that relies on a single aggregation would be a deep feature with a depth of 1, a feature that stacks two primitives would have a depth of 2 and so on.

**Automated Deep Feature Synthesis**

In addition to manually specifying aggregation and transformation feature primitives, we can let feature tools automatically generate many new features

In [0]:
# Perform deep feature synthesis without specifying primitives
features, feature_names = ft.dfs(entityset=es, target_entity='clients', 
                                 max_depth = 2)

In [114]:
features.head()

,income,credit_score,SUM(loans.loan_amount),SUM(loans.rate),STD(loans.loan_amount),STD(loans.rate),MAX(loans.loan_amount),MAX(loans.rate),SKEW(loans.loan_amount),SKEW(loans.rate),MIN(loans.loan_amount),MIN(loans.rate),MEAN(loans.loan_amount),MEAN(loans.rate),COUNT(loans),NUM_UNIQUE(loans.loan_type),NUM_UNIQUE(loans.repaid),MODE(loans.loan_type),MODE(loans.repaid),SUM(payments.payment_amount),STD(payments.payment_amount),MAX(payments.payment_amount),SKEW(payments.payment_amount),MIN(payments.payment_amount),MEAN(payments.payment_amount),COUNT(payments),NUM_UNIQUE(payments.missed),MODE(payments.missed),DAY(joined),YEAR(joined),MONTH(joined),WEEKDAY(joined),SUM(loans.STD(payments.payment_amount)),SUM(loans.MAX(payments.payment_amount)),SUM(loans.SKEW(payments.payment_amount)),SUM(loans.MIN(payments.payment_amount)),SUM(loans.MEAN(payments.payment_amount)),SUM(loans.NUM_UNIQUE(payments.missed)),STD(loans.SUM(payments.payment_amount)),STD(loans.MAX(payments.payment_amount)),...,SKEW(loans.SUM(payments.payment_amount)),SKEW(loans.STD(payments.payment_amount)),SKEW(loans.MAX(payments.payment_amount)),SKEW(loans.MIN(payments.payment_amount)),SKEW(loans.MEAN(payments.payment_amount)),SKEW(loans.COUNT(payments)),SKEW(loans.NUM_UNIQUE(payments.missed)),MIN(loans.SUM(payments.payment_amount)),MIN(loans.STD(payments.payment_amount)),MIN(loans.MAX(payments.payment_amount)),MIN(loans.SKEW(payments.payment_amount)),MIN(loans.MEAN(payments.payment_amount)),MIN(loans.COUNT(payments)),MIN(loans.NUM_UNIQUE(payments.missed)),MEAN(loans.SUM(payments.payment_amount)),MEAN(loans.STD(payments.payment_amount)),MEAN(loans.MAX(payments.payment_amount)),MEAN(loans.SKEW(payments.payment_amount)),MEAN(loans.MIN(payments.payment_amount)),MEAN(loans.MEAN(payments.payment_amount)),MEAN(loans.COUNT(payments)),MEAN(loans.NUM_UNIQUE(payments.missed)),NUM_UNIQUE(loans.MODE(payments.missed)),NUM_UNIQUE(loans.DAY(loan_start)),NUM_UNIQUE(loans.DAY(loan_end)),NUM_UNIQUE(loans.YEAR(loan_start)),NUM_UNIQUE(loans.YEAR(loan_end)),NUM_UNIQUE(loans.MONTH(loan_start)),NUM_UNIQUE(loans.MONTH(loan_end)),NUM_UNIQUE(loans.WEEKDAY(loan_start)),NUM_UNIQUE(loans.WEEKDAY(loan_end)),MODE(loans.MODE(payments.missed)),MODE(loans.DAY(loan_start)),MODE(loans.DAY(loan_end)),MODE(loans.YEAR(loan_start)),MODE(loans.YEAR(loan_end)),MODE(loans.MONTH(loan_start)),MODE(loans.MONTH(loan_end)),MODE(loans.WEEKDAY(loan_start)),MODE(loans.WEEKDAY(loan_end))
client_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
25707,211422,621,159279,69.54,4149.486062,2.484186,13913,9.44,-0.186352,0.735470,1212,0.33,7963.950000,3.477000,20,4,2,credit,0,189747,649.597374,2704,0.216726,124,1178.552795,161,2,1,6,2006,10,4,5618.597011,31575,2.192690,16301,23515.219922,40,5378.305752,800.026439,...,-0.024396,0.037431,-0.407373,0.190578,-0.115436,1.311640,0.0,1379,45.775656,232,-1.031769,172.375000,5,2,9487.350000,280.929851,1578.750000,0.109634,815.050000,1175.760996,8.050000,2.0,2,15,15,10,10,10,9,7,6,0,27,1,2010,2007,1,8,3,0
26326,227920,633,116321,40.28,4393.666631,2.057142,13464,6.73,0.149658,1.181651,1164,0.50,7270.062500,2.517500,16,4,2,other,0,155176,677.916482,2658,0.345898,119,1166.736842,133,2,1,6,2004,5,3,4321.316317,24760,-0.489554,13272,18587.065729,32,5494.901018,870.899458,...,-0.116116,2.065885,-0.181441,0.350923,-0.095161,1.773450,0.0,1118,29.236963,228,-1.035189,185.500000,5,2,9698.500000,270.082270,1547.500000,-0.030597,829.500000,1161.691608,8.312500,2.0,2,12,14,9,13,11,9,7,5,0,6,6,2003,2005,4,7,5,2
26695,174532,680,140845,44.39,4196.462499,1.561659,14865,6.51,0.168879,0.896574,2389,0.22,7824.722222,2.466111,18,4,2,home,0,164211,637.218815,2932,0.407758,259,1207.433824,136,2,0,27,2004,8,4,3752.677997,27339,0.337492,17008,21981.611064,36,5552.061496,754.768235,...,0.718539,0.382076,0.092412,0.695612,0.401835,2.254073,0.0,2215,69.535192,470,-1.777480,363.142857,5,2,9122.833333,208.482111,1518.833333,0.018750,944.888889,1221.200615,7.555556,2.0,2,12,13,12,11,8,11,7,6,